In [ ]:
import requests
import os
import scanpy as sc
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import gc


## Set parameters and paths

In [ ]:
# Define parameters
res = 1
n_hvg = 2000

In [ ]:
# Define the input and output directories
input_dir = 'Input_files'
output_dir = 'Output_Seeker_2023'

# Create the output directory if it does not exist
if not os.path.exists(input_dir):
    os.makedirs(input_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## Analysis for Seeker_2023_DS9

studyID = 'Seeker_2023'
download_link = 'https://datasets.cellxgene.cziscience.com/32c319ef-10e2-4948-8b40-093d2f9d7cb5.h5ad'
new_filename = f"Input_files/{studyID}.h5ad"
response = requests.get(download_link)
with open(new_filename, 'wb') as f:
        f.write(response.content)
print(f"File downloaded and saved as {new_filename}")

In [ ]:
studyID = 'Seeker_2023'
file = 'Seeker_2023.h5ad'
print(f'Start process {studyID}')
adata=sc.read_h5ad(os.path.join(input_dir,file))
print(f'    Before process: number of obs {adata.n_obs}, number of var {adata.n_vars}')

#adata.layers["counts"]=adata.X.copy()
adata.var['GeneID']=adata.var.index
adata.var.set_index('feature_name',inplace=True)
if adata.raw is not None:
    adata.raw.var.set_index('feature_name', inplace=True)
else:
    print("    adata.raw is None, skipping setting index.")
adata.var["mt"]=adata.var_names.str.startswith("MT-")

print(f'    Start QC')
sc.pp.calculate_qc_metrics(adata,qc_vars=["mt"],inplace=True,log1p=True)
sc.pp.filter_genes(adata,min_cells=3)
adata=adata[adata.obs['nFeature_RNA']>200,:]
adata=adata[adata.obs['pct_counts_mt']<10,:]
adata=adata[adata.obs['nCount_RNA']>400,:]
adata=adata[adata.obs['nCount_RNA']<60000,:]
print(f'    After process: number of obs {adata.n_obs}, number of var {adata.n_vars}')

sc.pp.highly_variable_genes(adata,n_top_genes=n_hvg)
sc.tl.pca(adata)
sc.pp.neighbors(adata,n_neighbors=10,n_pcs=30)
sc.tl.umap(adata)
sc.tl.leiden(adata,resolution=res,n_iterations=2)


In [ ]:
adata.obs['cell_type'].value_counts()

#### Figure 8A

In [ ]:
# Figure 8A
print(f'    Start to plot {studyID}')
sc.pl.umap(adata,color=['cell_type'],save = f'_{studyID}_allcelltypes_leiden_res{res}.pdf')

In [ ]:
# Figure 8A
from collections import OrderedDict
goi = []
gene_categories = {
    "microglia": ['BHLHE41', 'CX3CR1', 'P2RY12','TREM2', 'TMEM119', 'HEXB', 'SALL1'],
    "CAM": ['CD163', 'MRC1', 'LYVE1'],
    "oligodendrocytes": ["PLP1", "CNP"],
    "precursor_cells": ["PDGFRA", "PTPRZ1"],
    "astrocytes": ["GJA1", "GFAP"],    
    "excitatory_neurons": ["SNAP25", "SLC17A7"],
    "inhibitory_neurons": ["SNAP25", "GAD1"],
    "reelin_positive_neurons": ["SNAP25", "RELN"],
    "endothelial_cells_pericytes": ["CLDN5", "NOTCH3"],
    "immune_cells": ["HLA-A", "PTPRC"]
}
unique_genes = OrderedDict()
for genes in gene_categories.values():
    for gene in genes:
        unique_genes[gene] = None
        
goi = list(unique_genes.keys())
goi_in_adata = [gene for gene in goi if gene in adata.var_names]

# Define the new order of levels
new_order = ['microglial cell', 'central nervous system macrophage','oligodendrocyte', 'oligodendrocyte precursor cell', 'differentiation-committed oligodendrocyte precursor','astrocyte','neuron','GABAergic neuron','glutamatergic neuron','cerebellar granule cell','capillary endothelial cell','endothelial cell of artery','mural cell','vascular associated smooth muscle cell','leukocyte']
adata.obs['cell_type'] = pd.Categorical(adata.obs['cell_type'], categories=new_order, ordered=True)
print(adata.obs['cell_type'].cat.categories)

sc.pl.dotplot(adata, goi_in_adata, groupby="cell_type", standard_scale="var",title=f'Gene expression in each cell type in {studyID}',save=f'{studyID}_celltype.pdf')

#### Supplementary figure 14A

In [ ]:
# Importing necessary libraries
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt

# Define the markers you are interested in
markers = ['P2RY12','AIF1','C1QB','GPR34','CSF1R','SLCO2B1','TBXAS1','DOCK8','PCDH9','PLP1']

# Loop over each marker to create a density plot on UMAP
for marker in markers:
    # Create a UMAP scatter plot colored by marker expression
    sc.pl.umap(adata, color=marker, show=False, cmap='viridis', legend_loc=None,s=20)
    
    # Overlay density (this step is similar to Nebulosa, but done manually)
    ax = plt.gca()  # Get current axis
    
    # Extract UMAP coordinates
    x = adata.obsm['X_umap'][:, 0]  # UMAP 1 coordinates
    y = adata.obsm['X_umap'][:, 1]  # UMAP 2 coordinates
    
    # Extract expression levels of the marker
    expression = adata[:, marker].X.toarray().flatten()
    
   # Create a 2D histogram / density plot
    hexbin_plot = ax.hexbin(x, y, C=expression, gridsize=70, cmap='inferno', reduce_C_function=np.mean, mincnt=1)

    # Set axis labels and title
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    plt.title(f'Density plot for {marker}')

    # Add the colorbar with the mappable hexbin plot
    plt.colorbar(hexbin_plot, label='Density')
    
    # Save or show the figure
    plt.savefig(f'figures/umap_density_{studyID}_{marker}.pdf')
    plt.show()

#### Figure 8B

In [ ]:
# Importing necessary libraries
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt

# Define the markers you are interested in
markers = ["CD22",'BHLHE41']

# Loop over each marker to create a density plot on UMAP
for marker in markers:
    # Create a UMAP scatter plot colored by marker expression
    sc.pl.umap(adata, color=marker, show=False, cmap='viridis', legend_loc=None,s=20)
    
    # Overlay density (this step is similar to Nebulosa, but done manually)
    ax = plt.gca()  # Get current axis
    
    # Extract UMAP coordinates
    x = adata.obsm['X_umap'][:, 0]  # UMAP 1 coordinates
    y = adata.obsm['X_umap'][:, 1]  # UMAP 2 coordinates
    
    # Extract expression levels of the marker
    expression = adata[:, marker].X.toarray().flatten()
    
   # Create a 2D histogram / density plot
    hexbin_plot = ax.hexbin(x, y, C=expression, gridsize=70, cmap='inferno', reduce_C_function=np.mean, mincnt=1)

    # Set axis labels and title
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    plt.title(f'Density plot for {marker}')

    # Add the colorbar with the mappable hexbin plot
    plt.colorbar(hexbin_plot, label='Density')
    
    # Save or show the figure
    plt.savefig(f'figures/umap_density_{studyID}_{marker}.pdf')
    plt.show()

#### Extract microglia

In [ ]:
adata_m = adata[adata.obs['cell_type']=='microglial cell', :]
adata_m

#### Figure 8D

In [ ]:
# Figure 8D
res = 0.05
# Recluster microglia
sc.pp.highly_variable_genes(adata_m,n_top_genes=n_hvg)
sc.tl.pca(adata_m)
sc.pp.neighbors(adata_m,n_neighbors=10,n_pcs=30)
sc.tl.umap(adata_m)
sc.tl.leiden(adata_m,resolution=res,n_iterations=2)
sc.pl.umap(adata_m, color=['leiden'], palette='Paired',legend_loc='on data', save=f'_{studyID}_within_microglia_leiden_res{res}.pdf')    

In [ ]:
goi=['P2RY12','TMEM119','AIF1','C1QB','CX3R1','GPR34','CSF1R','SLCO2B1','TBXAS1','DOCK8','APBB1IP','PCDH9']

goi_in_adata=[gene for gene in goi if gene in adata_m.var_names]
sc.pl.dotplot(adata_m, goi_in_adata, groupby="leiden",save=f'{studyID}_within_microglia_goi_res{res}_check_microgliaMarker.pdf')

#### Figure 8E Plot CD22 and BHLE41

In [ ]:
# Importing necessary libraries
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt

# Define the markers you are interested in
markers = ["CD22",'BHLHE41']

# Loop over each marker to create a density plot on UMAP
for marker in markers:
    # Create a UMAP scatter plot colored by marker expression
    sc.pl.umap(adata_m, color=marker, show=False, cmap='viridis', legend_loc=None,s=20)
    
    # Overlay density (this step is similar to Nebulosa, but done manually)
    ax = plt.gca()  # Get current axis
    
    # Extract UMAP coordinates
    x = adata_m.obsm['X_umap'][:, 0]  # UMAP 1 coordinates
    y = adata_m.obsm['X_umap'][:, 1]  # UMAP 2 coordinates
    
    # Extract expression levels of the marker
    expression = adata_m[:, marker].X.toarray().flatten()
    
   # Create a 2D histogram / density plot
    hexbin_plot = ax.hexbin(x, y, C=expression, gridsize=70, cmap='inferno', reduce_C_function=np.mean, mincnt=1)

    # Set axis labels and title
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    plt.title(f'Density plot for {marker}')

    # Add the colorbar with the mappable hexbin plot
    plt.colorbar(hexbin_plot, label='Density')
    
    # Save or show the figure
    plt.savefig(f'figures/umap_density_{studyID}_{marker}_within_macrolia.pdf')
    plt.show()

In [ ]:
adata_m.obs['donor_id'].unique().size

In [ ]:
# PseudoBulk Analysis for BHLHE41 vs CD22 Exclusive Score between C0 and C1 in Microglia
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ranksums
import scanpy as sc

# Step 1: Compute Exclusive Score per cell

# Get dense expression (safe for two genes only)
bhlhe41 = np.log1p(adata_m[:, 'BHLHE41'].X.toarray().flatten())
cd22 = np.log1p(adata_m[:, 'CD22'].X.toarray().flatten())

adata_m.obs['BHLHE41–CD22 Tendency Score'] = np.log1p(bhlhe41) - np.log1p(cd22)


# Step 2: Pseudobulk = donor × cluster summary
# We compute mean Exclusive score per donor per cluster
df = adata_m.obs[['BHLHE41–CD22 Tendency Score', 'leiden', 'donor_id']].copy()
df = df.dropna(subset=['BHLHE41–CD22 Tendency Score'])

pseudobulk = (
    df.groupby(['donor_id', 'leiden'])['BHLHE41–CD22 Tendency Score']
      .mean()
      .reset_index()
)
pseudobulk = pseudobulk.dropna(subset=['BHLHE41–CD22 Tendency Score'])

print("\nPseudobulk table:")
print(pseudobulk)

# Step 3: Prepare data for C0 vs C1 comparison
cluster0 = pseudobulk[pseudobulk['leiden'] == '0']['BHLHE41–CD22 Tendency Score']
cluster1 = pseudobulk[pseudobulk['leiden'] == '1']['BHLHE41–CD22 Tendency Score']

print(f"\nDonor counts → C0: {len(cluster0)}, C1: {len(cluster1)}")

# Step 4: Statistical testing (Wilcoxon rank-sum on donors)
statistic, p_value = ranksums(cluster0, cluster1)

print("\n--- Wilcoxon Rank-Sum Test (Pseudobulk Donor-level) ---")
print(f"Cluster 0 vs Cluster 1:")
print(f"Statistic = {statistic:.2f}")
print(f"P = {p_value:.3e}")

if p_value < 0.05:
    print("Conclusion: Significant donor-level difference between clusters.")
else:
    print("Conclusion: No significant donor-level difference between clusters.")


# Step 5: Plot donor-level pseudobulk results
plt.figure(figsize=(4,4))
plt.boxplot([cluster0, cluster1], labels=['C0', 'C1'])
plt.ylabel("Pseudobulk Exclusive Score")
plt.title("Donor-level Pseudobulk Comparison")
plt.show()

import seaborn as sns
import matplotlib.pyplot as plt

# violin plot
plot_df = pseudobulk[['leiden', 'BHLHE41–CD22 Tendency Score']].copy()
plot_df['leiden'] = plot_df['leiden'].astype(str)   # ensure string labels

plt.figure(figsize=(6,8))
sns.violinplot(data=plot_df, x='leiden', y='BHLHE41–CD22 Tendency Score', inner='box')
plt.xlabel("Cluster")
plt.ylabel("Pseudobulk Exclusive Score")
plt.title("Donor-level Pseudobulk Violin Plot")
plt.savefig(f'figures/{studyID}_Violin_{GENE_X}_vs_{GENE_Y}_cluster.pdf')
plt.show()
